In [8]:
import unittest
import time
import sys
import traceback
import requests

from console_progressbar import ProgressBar
from bs4 import BeautifulSoup
from lxml import etree

#test
def list_chess_players():
    try:
        BASE_URL = "https://www.chessgames.com" 
        cnt = 1
        players = [] 
    
        time.sleep(3)
        tic = time.perf_counter()
        link = BASE_URL
        reps = requests.get(link)
        toc = time.perf_counter()
            
        #leer el contenido de la pagina
        html = reps.content
        if (html):  
            soup = BeautifulSoup(html,'html.parser')
            plyer_select = soup.find("select",attrs={"name":"pid"})
            options = plyer_select.find_all("option")
            for x in options:
                #print(x['value'] +' '+ x.text)
                if (x['value']):
                    players.append({'code':x['value'],'name':x.text})
 
        return players         
    except ValueError:
            print(f"Error de Valor :{link}")
           # return games                     
            traceback.print_exception(*sys.exc_info())
    except:    
            print(f"Error general :{link}") 
            #return games 
            traceback.print_exception(*sys.exc_info())  

def list_chess_game(param):
    try:
        BASE_URL = "https://www.chessgames.com"
        year=param['year']
        playercomp=param['playercomp']
        pid=param['pid']
        result=param['result']
        link = f"{BASE_URL}/perl/chess.pl?yearcomp=exactly&year={year}&playercomp={playercomp}&pid={pid}&result={result}"
        cnt = 1
        games = [] 
    
        time.sleep(3)
        tic = time.perf_counter()
        reps = requests.get(link)
        toc = time.perf_counter()
            
        #leer el contenido de la pagina
        html = reps.content
        if (html):  
            soup = BeautifulSoup(html,'html.parser')
            tbs_parent = soup.find_all("table")
            tbs_child  = tbs_parent[1].find_all("table")
            rows   = tbs_child[1].find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                alla  = cols[0].find_all('a')
                for a in alla:
                    games.append(f"{BASE_URL}{a.attrs.get('href')}")
                                     
        return games          
    except ValueError:
            #print(f"Error de Valor :{link}")
            #traceback.print_exception(*sys.exc_info())
            return games    
    except:    
            #print(f"Error general :{link}") 
            #traceback.print_exception(*sys.exc_info())
            return games    

def Load_write_game(link):
    
    #logging.basicConfig(filename='example.log', encoding='utf-8', level=logging.DEBUG)
    try:
       #carga el archivo de los datos
        file = open("chess_game.txt","a")
        gid = 1000000
        cnt = 1
        pgns = []
        
        # cargar la pagina
        time.sleep(3)
        tic = time.perf_counter()
        reps = requests.get(link)
        toc = time.perf_counter()
            
        #leer el contenido de la pagina
        html = reps.content
        if (html):  
            #loggin.info(f"Process {toc - tic+1:0.4} seconds url: {link}")
            soup = BeautifulSoup(html,'html.parser')
            olga_data = soup.find_all("div", attrs={"id":"olga-data"})
            
            #Carga data
            if (olga_data): 
               pgn = olga_data[0].attrs.get('pgn')
               line = pgn.replace('\n',' ').replace('\r',' ')+'\n'
               file.write(line) 
            else :
              print("Error de contenido")

        file.close()    
    except ValueError:
            file.close()
            print("Error de Valor")
            traceback.print_exception(*sys.exc_info())
    except:    
            file.close()
            print("Error de general") 
            traceback.print_exception(*sys.exc_info())
#1475
pyc = 1
rango_year  = range(2019,2021 + 1)
rango_color = [{'color':'white','result':'1-0'},{'color':'black','result':'0-1'}]

for year in rango_year:
    for cl_res in rango_color:       
        pt    = 1      
        param = {'year':str(year),'playercomp':cl_res['color'],'pid':'52948','result':cl_res['result']}                                 
        gameslinks = list_chess_game(param)
        cnt   = len(gameslinks)
        message = f"DOWL {str(year)} {cl_res['color']} {cl_res['result']}"
        pb = ProgressBar(total=cnt,prefix= message, suffix=' ',decimals=2,length=50,fill='▐',zfill=' ')
        for game in gameslinks:
            #print(game)  
            Load_write_game(game)
            pb.print_progress_bar(pt)
            pt = pt + 1

#print(gameslinks)                   
#print("Proceso finalizado")

#print(list_chess_players())            
                                 
                                 